## Decomposition（クエリ分解）

- 与えられた質問を複数のサブクエリに分解
- サブクエリの回答を組み合わせて元の質問の回答を得る
- サブクエリを順番に処理して、次のサブクエリに前のサブクエリの結果を渡すケース（Perplexity.AI）
- サブクエリを並列に処理するケースがある（Genspark）

In [10]:
# Load blog
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader(
    web_paths=("https://zenn.dev/knowledgesense/articles/47de9ead8029ba",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("Container_wide__ykGLh Container_common__figYY")
        )
    ),
)
blog_docs = loader.load()

# Split
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=500, 
    chunk_overlap=50)

# Make splits
splits = text_splitter.split_documents(blog_docs)

# Index
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [1]:
from langchain.prompts import ChatPromptTemplate

template = """あなたは、入力された質問に関連する複数のサブ質問を生成する役に立つアシスタントです。
目的は、入力された質問を、個別に回答できる一連のサブ問題やサブ質問に分解することです。
次の質問に関連する複数の検索クエリを生成してください。

質問: {question}

出力（3つのクエリ）:"""

prompt_decomposition = ChatPromptTemplate.from_template(template)

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

llm = ChatOpenAI(temperature=0)

generate_queries_decomposition = (
    prompt_decomposition
    | llm
    | StrOutputParser()
    | (lambda x: x.split("\n"))
)

In [5]:
generate_queries_decomposition.invoke({"question": "RAGとはなんですか？"})

['1. RAGの略は何ですか？', '2. RAGはどのような分野で使用される言葉ですか？', '3. RAGの歴史や起源について何か情報がありますか？']

In [6]:
generate_queries_decomposition.invoke({"question": "アカウントを削除したいです"})

['1. アカウント削除手順', '2. アカウント削除の注意事項', '3. アカウント削除後のデータの取り扱い']

In [7]:
generate_queries_decomposition.invoke({"question": "ログインできなくなりました"})

['1. ログイン画面に表示されるエラーメッセージは何ですか？',
 '2. 最後にパスワードを変更した日付はわかりますか？',
 '3. ログインできなくなったアカウントは、他のデバイスからもアクセスできない状態ですか？']

In [9]:
template = """こちらがあなたが回答すべき質問です:
{question}

こちらは利用可能な背景質問+回答のペアです:
{q_a_pairs}

こちらは質問に関連する追加のコンテキストです:
{context}

上記のコンテキストと背景の質問+回答ペアを使用して、次の質問に回答してください:
{question}"""

decomposition_prompt = ChatPromptTemplate.from_template(template)

In [11]:
from operator import itemgetter
from langchain_core.output_parsers import StrOutputParser


def format_qa_pair(question, answer):
    formatted_string = ""
    formatted_string += f"質問: {question}\n回答: {answer}\n\n"
    return formatted_string


llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

questions = generate_queries_decomposition.invoke({"question": "RAGとはなんですか？"})
print(questions)

q_a_pairs = ""

# 1つずつ順番に質問に回答
for q in questions:
    rag_chain = (
        {"context": itemgetter("question") | retriever,
         "question": itemgetter("question"),
         "q_a_pairs": itemgetter("q_a_pairs")}
        | decomposition_prompt
        | llm
        | StrOutputParser()
    )

    answer = rag_chain.invoke({"question": q, "q_a_pairs": q_a_pairs})

    # 前のクエリの質問と回答のペアを追加
    q_a_pair = format_qa_pair(q, answer)
    q_a_pairs = q_a_pairs + "\n" + q_a_pair

['1. RAGの略称は何ですか？', '2. RAGはどのような分野で使用される言葉ですか？', '3. RAGの歴史や起源について知りたいです。']


In [13]:
print(answer)

RAG（Retrieval-Augmented Generation）の歴史や起源については、具体的な年や発表に関する詳細な情報は限られていますが、RAGの概念は、自然言語処理（NLP）や人工知能（AI）の進化の中で生まれました。

RAGは、従来の大規模言語モデル（LLM）の限界を克服するために開発された手法の一つです。従来のLLMは、学習データに基づいて生成された回答を提供するため、事実と異なる情報を生成する「ハルシネーション」や、学習データに含まれていない情報に対する回答ができないという問題がありました。これに対処するために、外部データベースから関連するドキュメントを取得し、それを基にして回答を生成するRAGのアプローチが提案されました。

RAGの登場により、情報検索や質問応答システムにおいて、より正確で信頼性の高い情報提供が可能になりました。この手法は、特にビジネスや教育、カスタマーサポートなどの実務の場面での応用が進んでいます。

RAGの具体的な技術的な発展や研究は、近年の生成AIの進化とともに進んでおり、今後もさらなる研究や実装が期待されています。


In [17]:
# クエリを並列に検索する場合
from langchain import hub
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

prompt_rag = hub.pull("rlm/rag-prompt")

def retrieve_and_rag(question, prompt_rag, sub_question_generator_chain):
    # decompositionでクエリを分解
    sub_questions = sub_question_generator_chain.invoke({"question": question})

    rag_results = []
    for sub_question in sub_questions:
        retrieved_docs = retriever.get_relevant_documents(sub_question)
        answer = (prompt_rag | llm | StrOutputParser()).invoke({"context": retrieved_docs, "question": sub_question})
        rag_results.append(answer)

    return rag_results, sub_questions

In [19]:
question = "RAGとはなんですか？"

answers, questions = retrieve_and_rag(question, prompt_rag, generate_queries_decomposition)

In [20]:
answers

['RAGの略は「Retrieval-Augmented Generation」です。これは、外部データベースから関連するドキュメントを取得し、それを基に回答を生成する方法です。RAGは、通常のLLMの限界を克服するために必要とされています。',
 'RAGは、主に情報検索や自然言語処理の分野で使用される言葉です。特に、ユーザーの質問に対して関連するドキュメントを参照し、正確な回答を生成するための手法として重要です。これにより、従来の言語モデルの限界を克服することが可能になります。',
 'RAG（Retrieval-Augmented Generation）は、外部データベースから関連するドキュメントを取得し、それを基に回答を生成するLLMの手法です。この技術は、通常のLLMの限界を克服し、正確な情報を提供するために必要とされています。具体的な歴史や起源についての詳細は、提供された文脈には含まれていません。']

In [21]:
questions

['1. RAGの略は何ですか？', '2. RAGはどのような分野で使用される言葉ですか？', '3. RAGの歴史や起源について知りたいです。']

In [27]:
def format_qa_pairs(questions, answers):
    """Format Q and A pairs"""
    formatted_string = ""
    for i, (question, answer) in enumerate(zip(questions, answers), start=1):
        formatted_string += f"質問 {i}: {question}\n回答 {i}: {answer}\n\n"
    return formatted_string.strip()


context = format_qa_pairs(questions, answers)
print(context)

質問 1: 1. RAGの略は何ですか？
回答 1: RAGの略は「Retrieval-Augmented Generation」です。これは、外部データベースから関連するドキュメントを取得し、それを基に回答を生成する方法です。RAGは、通常のLLMの限界を克服するために必要とされています。

質問 2: 2. RAGはどのような分野で使用される言葉ですか？
回答 2: RAGは、主に情報検索や自然言語処理の分野で使用される言葉です。特に、ユーザーの質問に対して関連するドキュメントを参照し、正確な回答を生成するための手法として重要です。これにより、従来の言語モデルの限界を克服することが可能になります。

質問 3: 3. RAGの歴史や起源について知りたいです。
回答 3: RAG（Retrieval-Augmented Generation）は、外部データベースから関連するドキュメントを取得し、それを基に回答を生成するLLMの手法です。この技術は、通常のLLMの限界を克服し、正確な情報を提供するために必要とされています。具体的な歴史や起源についての詳細は、提供された文脈には含まれていません。


In [30]:
template = """ここに質問と回答のペアがあります:

{context}

これらを使って、次の質問に対する答えをまとめてください: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
final_rag_chain = (
    prompt
    | llm
    | StrOutputParser()
)

print(final_rag_chain.invoke({"context": context, "question": question}))

RAG（Retrieval-Augmented Generation）とは、外部データベースから関連するドキュメントを取得し、それを基に回答を生成する手法です。この技術は、情報検索や自然言語処理の分野で主に使用されており、ユーザーの質問に対して正確な回答を提供するために重要です。RAGは、従来の言語モデルの限界を克服するために開発され、より正確な情報を提供することを目的としています。
